In [1]:
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_df  = pd.read_csv("/kaggle/input/hell-week/train.csv")
test_df = pd.read_csv("/kaggle/input/hell-week/test.csv")
for x in train_df.columns[10:-1]:
    train_df[x] = train_df[x].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)
    
repeating_columns = []
for col in train_df.columns[10:-1]:
    if len(train_df[col].unique()) == 2:
        repeating_columns.append(col)
train_df = train_df.drop(columns=repeating_columns,axis=1)
train_df = train_df.dropna()
features = train_df.columns[10:-1]
X = train_df[features]
X = X.dropna()


for x in test_df.columns[10:]:
    test_df[x] = test_df[x].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)
X_test = test_df[features]

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = train_df['state']
y_encoded = label_encoder.fit_transform(y)
y_encoded_onehot = np.eye(10)[y_encoded]
print(y_encoded_onehot)



X_train, X_val, y_train, y_val = train_test_split(X, y_encoded_onehot, test_size=0.3)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

num_unique_categories = y_encoded_onehot.shape[1]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
(40574, 55)
(40574, 10)
(17389, 55)
(17389, 10)


In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_unique_categories, activation='softmax')])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))
predictions = model.predict(X_test)
y_pred_labels = np.argmax(predictions, axis=1)
y_pred_decoded = label_encoder.inverse_transform(y_pred_labels)
output = pd.DataFrame({'UID': test_df.UID, 'state':y_pred_decoded})
output.to_csv('submission.csv', index=False)

Epoch 1/20
1268/1268 [==============================] - 5s 3ms/step - loss: 2.3724 - accuracy: 0.2198 - val_loss: 1.9039 - val_accuracy: 0.3297
Epoch 2/20
1268/1268 [==============================] - 3s 2ms/step - loss: 1.6201 - accuracy: 0.4400 - val_loss: 1.4119 - val_accuracy: 0.5152
Epoch 3/20
1268/1268 [==============================] - 3s 3ms/step - loss: 1.2690 - accuracy: 0.5678 - val_loss: 1.1921 - val_accuracy: 0.5898
Epoch 4/20
1268/1268 [==============================] - 4s 3ms/step - loss: 1.0832 - accuracy: 0.6305 - val_loss: 1.0577 - val_accuracy: 0.6395
Epoch 5/20
1268/1268 [==============================] - 3s 2ms/step - loss: 0.9656 - accuracy: 0.6726 - val_loss: 0.9928 - val_accuracy: 0.6659
Epoch 6/20
1268/1268 [==============================] - 3s 2ms/step - loss: 0.8851 - accuracy: 0.6996 - val_loss: 0.9249 - val_accuracy: 0.6886
Epoch 7/20
1268/1268 [==============================] - 3s 2ms/step - loss: 0.8209 - accuracy: 0.7238 - val_loss: 0.8644 - val_accuracy: